In [0]:
%run ../config/setup

# Gold Layer ETL Process

## Why I Made This Notebook
I want to turn my cleaned energy demand data into business-focused tables that make analysis easy. By building these gold tables, I can answer important questions about energy use and help others make decisions based on the data.

## Gold Table 1: Hourly Energy Consumption Summary
I group the data by hour of the day and calculate average consumption, minimum and maximum voltage, and average current intensity. This helps me spot peak load hours and understand how energy use changes throughout the day.

**Columns:**
* hour_of_day
* avg_consumption_kwh
* min_voltage
* max_voltage
* avg_current_intensity

## Gold Table 2: Daily Consumption Trends
I group the data by date and calculate total and average energy use, as well as the peak load. I also plan to add columns for the peak hour and whether it's a weekend, so I can see day-wise patterns and off-peak trends.

**Columns:**
* date
* total_kwh
* avg_kwh
* peak_hour
* weekend_flag

## Gold Table 3: Sub-Meter Usage Breakdown
I group the data by date and sum up the values for each sub-meter. This lets me analyze how much energy is used in the kitchen, laundry, and water heating areas.

**Columns:**
* date
* sub_meter_1_total
* sub_meter_2_total
* sub_meter_3_total

## What This Setup Achieves
By creating these gold tables, I make it easy to:
* Find peak hours and daily trends
* Compare energy use across different parts of the house
* Support business decisions with clear, organized data

This setup helps me turn raw numbers into useful insights for anyone interested in energy demand.

In [0]:
from pyspark.sql.functions import avg, sum, min, max, col

df_silver = spark.table(full_path_silver)



In [0]:
# Table 1: hourly
# Purpose: Identify peak load hours.
df_hourly = df_silver.groupBy("hour_of_day") \
    .agg(
        avg("consumption_kwh").alias("avg_consumption_kwh"),
        min("voltage").alias("min_voltage"),
        max("voltage").alias("max_voltage"),
        avg("global_intensity").alias("avg_current_intensity")
    ).orderBy("hour_of_day")
df_hourly.write.format("delta").mode("overwrite").saveAsTable(full_path_gold_hourly)

# display a few values
display(df_hourly.limit(25))

In [0]:
# Table 2: Daily
# Purpose: Understand day-wise variations & off-peak patterns.
df_daily = df_silver.groupBy(col("datetime").cast("date").alias("date")) \
    .agg(
        sum("consumption_kwh").alias("total_kwh"),
        avg("consumption_kwh").alias("avg_kwh"),
        max("consumption_kwh").alias("peak_load")
    ).orderBy("date")

df_daily.write.format("delta").mode("overwrite").saveAsTable(full_path_gold_daily)

# display a few values
display(df_daily.limit(25))

In [0]:
# Table 3: Sub-Meters
# Purpose: Analyze kitchen, laundry, and water heating consumption.
df_submeter = df_silver.groupBy(col("datetime").cast("date").alias("date")) \
    .agg(
        sum("Sub_metering_1").alias("sub_meter_1_total"),
        sum("Sub_metering_2").alias("sub_meter_2_total"),
        sum("Sub_metering_3").alias("sub_meter_3_total")
    ).orderBy("date")

df_submeter.write.format("delta").mode("overwrite").saveAsTable(full_path_gold_submeter)

# display a few values
display(df_submeter.limit(25))